# Processing thread for XMM-Newton EPIC observations

## Step 02, tailored for PN Large Window mode

The following is happening here:

1. Sets the folders where the ODF is and where the products will be saved
2. Checks if the necessary SAS files are available: ccf.cif and sets some environmental variables
3. Checks if the observation is with the required mode, e.g. Large Window
4. Runs epchain for the processing


In [10]:
import os
import subprocess
import sys
import logging
import glob

In [11]:
# function to run shell process
# get the SAS version, will be used for the output folder 
def run_command(command):
    try:
        result = subprocess.run([command], shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
        retcode = result.returncode
        if retcode < 0:
            print("Warning: \"{}\" was terminated by signal {} \n {}".format(command,-retcode,result.stdout.decode()))
            logging.warning("\"{}\" was terminated by signal {} \n {}".format(command,-retcode,result.stdout.decode()))
        else:
            print("Info: \"{}\" returned {} \n {}".format(command,retcode,result.stdout.decode()))
            logging.info("\"{}\" returned {} \n {}".format(command,retcode,result.stdout.decode()))
    except OSError as e:
        print("Execution of \"{}\" failed:".format(command), e, file=sys.stderr)
        logging.error("Execution of \"{}\" failed:".format(command), e, file=sys.stderr)
    return retcode

In [12]:
os.environ['SAS_CCFPATH'] = '/xdata/ccf/pub'
#obsid = "0721010501"
obsid = "0721010401"

home = os.path.expanduser('~')
wdir = f"{home}/IVAN/Cu-line"

if (not os.path.isdir(wdir)):
    print (f"The input working folder {wdir} does not exist.")
    raise FileNotFoundError
#
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(levelname)s %(message)s',
                    filename=f'{wdir}/{obsid}_proc_step02.log',
                    filemode='w')
# the ODF directory
odfDir = os.path.join(wdir,obsid)
if (not os.path.isdir(odfDir)):
    print (f"The ODF folder {odfDir} does not exist! Cannot continue!")
    logging.error (f"The ODF folder {odfDir} does not exist! Cannot continue!")
    raise FileNotFoundError
#
ppsDir = os.path.join(odfDir,"PN_LW")
#
if (not os.path.isdir(ppsDir)):
    print (f"PPS folder {ppsDir} does not exist. Will create it.")
    logging.warning (f"PPS folder {ppsDir} does not exist. Will create it.")
    os.mkdir(ppsDir)
#

PPS folder /home/ivaltchanov/IVAN/Cu-line/0721010401/PN_LW does not exist. Will create it.


In [13]:
os.environ["SAS_ODF"] = odfDir
ccffile = os.path.join(odfDir,"ccf.cif")
if (not os.path.isfile(ccffile)):
    print (f"No ccf.cif file found in ODF folder {odfDir}")
    print ("     ===> This means epic_proc_step01 was not run?")
    logging.error(f"No CCF file ccf.cif found in {odfDir}")
    raise FileNotFoundError
#
os.environ["SAS_CCF"] = ccffile
logging.info(f"Set SAS_ODF to {odfDir}")
logging.info(f"Set SAS_CCF to {ccffile}")
#

In [14]:
#
# parse the ODF Summary SAS file to only select observations with SmallWindow mode for PN
#
mode = "PrimeLargeWindow"
goodMode = False
sumFile = glob.glob("{}/*.SAS".format(odfDir))

if (len(sumFile) != 1):
    print (f"No ODF summary file *.SAS found in ODF folder {odfDir}")
    print ("     ===> This means epic_proc_step01 was not run?")
    logging.error(f"No ODF summary file *.SAS found in {odfDir}")
    raise FileNotFoundError
#
with open(sumFile[0],'r') as sas:
    lines = sas.readlines()
for qline in lines:
    if (f"MODE = {mode}" in qline):
        goodMode = True
        break
#
if (not goodMode):
    print (f"Not processing {obsid} as it is not with PN {mode}")
    logging.error(f"Observation {obsid} not with PN {mode}")
    raise Exception
else:
    print (f"Processing {obsid} as it is with PN {mode}")
    logging.info(f"Observation {obsid} is with {mode}")
#
#

Processing 0721010401 as it is with PN PrimeLargeWindow


In [15]:
#
# ======== EPCHAIN =============
#
os.chdir(ppsDir)
#
task = "epchain"
xtask = f"{task} odf={odfDir} odfaccess=all backgroundtres=N -V 1"
print ("*** RUNNING %s"%xtask)
status = run_command(xtask)
if (status != 0):
    raise Exception
print (f"*** all done for {obsid} and {task}")
logging.info (f"*** all done for {obsid} and {task}")
#

*** RUNNING epchain odf=/home/ivaltchanov/IVAN/Cu-line/0721010401 odfaccess=all backgroundtres=N
Info: "epchain odf=/home/ivaltchanov/IVAN/Cu-line/0721010401 odfaccess=all backgroundtres=N" returned 0 
 -:- -------------------------------------------------------------------
-:- ARGV = ! odf=/home/ivaltchanov/IVAN/Cu-line/0721010401 odfaccess=all backgroundtres=N ! 

-:- Test: TIME RAWX RAWY DETX DETY X Y PHA PI FLAG PATTERN PAT_ID PAT_SEQ OFFSETX
-:- Tested: TIME RAWX RAWY DETX DETY X Y PHA PI FLAG PATTERN PAT_ID PAT_SEQ OFFSETX
-:- Test: double int16 int16 int16 int16 int32 int32 int16 int16 int32 int8 int16 int8 int16
-:- Tested: double int16 int16 int16 int16 int32 int32 int16 int16 int32 int8 int16 int8 int16
-:-  EXPOSURE = 1
-:-  1 IMAGING exposure in ODF:
-:-  1: 2522_0721010401_PNS003 IM
-:-  
-:- CMD: atthkgen atthkset=P0721010401OBX000ATTTSR0000.FIT timestep=1 -w 10
atthkgen:- Executing (routine): atthkgen atthkset=P0721010401OBX000ATTTSR0000.FIT timestep=1 timebegin=0 timeen